<a href="https://colab.research.google.com/github/Swinden/Study/blob/main/demo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import zipfile

def unzip_file(zip_file_path, output_folder_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(output_folder_path)

zip_file_path = '/content/基于论文摘要的文本分类与关键词抽取挑战赛公开数据.zip'  # 需要解压的zip文件路径
output_folder_path = '/content/'  # 解压后文件存放的文件夹路径

unzip_file(zip_file_path, output_folder_path)


In [4]:
# 导入前置依赖
import os
from pathlib import Path

import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

# 用于加载bert模型
# 用于加载bert模型的分词器
from transformers import AutoTokenizer, BertModel

batch_size = 16
# 文本的最大长度
text_max_length = 128
# 总训练的epochs数，我只是随便定义了个数
epochs = 30
# 学习率
lr = 3e-5
# 取多少训练集的数据作为验证集
validation_ratio = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 每多少步，打印一次loss
log_per_step = 50

# 数据集所在位置
dataset_dir = Path("/content/data")
os.makedirs(dataset_dir) if not os.path.exists(dataset_dir) else ""

# 模型存储路径
model_dir = Path("/content/model/bert_checkpoints")
# 如果模型目录不存在，则创建一个
os.makedirs(model_dir) if not os.path.exists(model_dir) else ""

print("Device:", device)

# 读取数据集，进行数据处理

pd_train_data = pd.read_csv("/content/data/train.csv")
pd_train_data["title"] = pd_train_data["title"].fillna("")
pd_train_data["abstract"] = pd_train_data["abstract"].fillna("")

test_data = pd.read_csv("/content/data/test.csv")
test_data["title"] = test_data["title"].fillna("")
test_data["abstract"] = test_data["abstract"].fillna("")
pd_train_data["text"] = (
    pd_train_data["title"].fillna("")
    + " "
    + pd_train_data["author"].fillna("")
    + " "
    + pd_train_data["abstract"].fillna("")
    + " "
    + pd_train_data["Keywords"].fillna("")
)
test_data["text"] = (
    test_data["title"].fillna("")
    + " "
    + test_data["author"].fillna("")
    + " "
    + test_data["abstract"].fillna("")
    + " "
    + pd_train_data["Keywords"].fillna("")
)


# 从训练集中随机采样测试集
validation_data = pd_train_data.sample(frac=validation_ratio)
train_data = pd_train_data[~pd_train_data.index.isin(validation_data.index)]


# 构建Dataset
class MyDataset(Dataset):
    def __init__(self, mode="train"):
        super(MyDataset, self).__init__()
        self.mode = mode
        # 拿到对应的数据
        if mode == "train":
            self.dataset = train_data
        elif mode == "validation":
            self.dataset = validation_data
        elif mode == "test":
            # 如果是测试模式，则返回内容和uuid。拿uuid做target主要是方便后面写入结果。
            self.dataset = test_data
        else:
            raise Exception("Unknown mode {}".format(mode))

    def __getitem__(self, index):
        # 取第index条
        data = self.dataset.iloc[index]
        # 取其内容
        text = data["text"]
        # 根据状态返回内容
        if self.mode == "test":
            # 如果是test，将uuid做为target
            label = data["uuid"]
        else:
            label = data["label"]
        # 返回内容和label
        return text, label

    def __len__(self):
        return len(self.dataset)


train_dataset = MyDataset("train")
validation_dataset = MyDataset("validation")


train_dataset.__getitem__(0)

# 获取Bert预训练模型
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


# 接着构造我们的Dataloader。
# 我们需要定义一下collate_fn，在其中完成对句子进行编码、填充、组装batch等动作：
def collate_fn(batch):
    """
    将一个batch的文本句子转成tensor，并组成batch。
    :param batch: 一个batch的句子，例如: [('推文', target), ('推文', target), ...]
    :return: 处理后的结果，例如：
             src: {'input_ids': tensor([[ 101, ..., 102, 0, 0, ...], ...]), 'attention_mask': tensor([[1, ..., 1, 0, ...], ...])}
             target：[1, 1, 0, ...]
    """
    text, label = zip(*batch)
    text, label = list(text), list(label)

    # src是要送给bert的，所以不需要特殊处理，直接用tokenizer的结果即可
    # padding='max_length' 不够长度的进行填充
    # truncation=True 长度过长的进行裁剪
    src = tokenizer(
        text,
        padding="max_length",
        max_length=text_max_length,
        return_tensors="pt",
        truncation=True,
    )

    return src, torch.LongTensor(label)


train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn
)
validation_loader = DataLoader(
    validation_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn
)

inputs, targets = next(iter(train_loader))
print("inputs:", inputs)
print("targets:", targets)


# 定义预测模型，该模型由bert模型加上最后的预测层组成
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        # 加载bert模型
        self.bert = BertModel.from_pretrained("bert-base-uncased", mirror="tuna")

        # 最后的预测层
        self.predictor = nn.Sequential(
            nn.Linear(768, 256), nn.ReLU(), nn.Linear(256, 1), nn.Sigmoid()
        )

    def forward(self, src):
        """
        :param src: 分词后的推文数据
        """

        # 将src直接序列解包传入bert，因为bert和tokenizer是一套的，所以可以这么做。
        # 得到encoder的输出，用最前面[CLS]的输出作为最终线性层的输入
        outputs = self.bert(**src).last_hidden_state[:, 0, :]

        # 使用线性层来做最终的预测
        return self.predictor(outputs)


model = MyModel()
model = model.to(device)

# 定义出损失函数和优化器。这里使用Binary Cross Entropy：
criteria = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


# 由于inputs是字典类型的，定义一个辅助函数帮助to(device)
def to_device(dict_tensors):
    result_tensors = {}
    for key, value in dict_tensors.items():
        result_tensors[key] = value.to(device)
    return result_tensors


# 定义一个验证方法，获取到验证集的精准率和loss
def validate():
    model.eval()
    total_loss = 0.0
    total_correct = 0
    for inputs, targets in validation_loader:
        inputs, targets = to_device(inputs), targets.to(device)
        outputs = model(inputs)
        loss = criteria(outputs.view(-1), targets.float())
        total_loss += float(loss)

        correct_num = (((outputs >= 0.5).float() * 1).flatten() == targets).sum()
        total_correct += correct_num

    return total_correct / len(validation_dataset), total_loss / len(validation_dataset)


# 首先将模型调成训练模式
model.train()

# 清空一下cuda缓存
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# 定义几个变量，帮助打印loss
total_loss = 0.0
# 记录步数
step = 0

# 记录在验证集上最好的准确率
best_accuracy = 0

# 开始训练
for epoch in range(epochs):
    model.train()
    for i, (inputs, targets) in enumerate(train_loader):
        # 从batch中拿到训练数据
        inputs, targets = to_device(inputs), targets.to(device)
        # 传入模型进行前向传递
        outputs = model(inputs)
        # 计算损失
        loss = criteria(outputs.view(-1), targets.float())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += float(loss)
        step += 1

        if step % log_per_step == 0:
            print(
                "Epoch {}/{}, Step: {}/{}, total loss:{:.4f}".format(
                    epoch + 1, epochs, i, len(train_loader), total_loss
                )
            )
            total_loss = 0

        del inputs, targets

    # 一个epoch后，使用过验证集进行验证
    accuracy, validation_loss = validate()
    print(
        "Epoch {}, accuracy: {:.4f}, validation loss: {:.4f}".format(
            epoch + 1, accuracy, validation_loss
        )
    )
    # torch.save(model, model_dir / f"model_{epoch}.pt")

    # 保存最好的模型
    if accuracy > best_accuracy:
        torch.save(model, model_dir / f"model_best.pt")
        best_accuracy = accuracy

# 加载最好的模型，然后进行测试集的预测
model = torch.load(model_dir / f"model_best.pt")
model = model.eval()


test_dataset = MyDataset("test")
test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn
)


results = []
for inputs, ids in test_loader:
    outputs = model(inputs.to(device))
    outputs = (outputs >= 0.5).int().flatten().tolist()
    ids = ids.tolist()
    results = results + [(id, result) for result, id in zip(outputs, ids)]


test_label = [pair[1] for pair in results]
test_data["label"] = test_label
test_data[["uuid", "Keywords", "label"]].to_csv("submit_task1.csv", index=None)

Device: cuda


inputs: {'input_ids': tensor([[  101, 25930,  1011,  ...,  1037,  3451,   102],
        [  101,  3117,  3921,  ...,  2800, 12148,   102],
        [  101,  1037,  2382,  ...,  3591,  2044,   102],
        ...,
        [  101,  1037,  7721,  ...,  2671,  1010,   102],
        [  101,  2482,  4140,  ...,  1037,  3435,   102],
        [  101,  2969,  1011,  ...,  1996,  5579,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}
targets: tensor([0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0])


Epoch 1/30, Step: 49/300, total loss:13.2536
Epoch 1/30, Step: 99/300, total loss:6.4946
Epoch 1/30, Step: 149/300, total loss:7.7485
Epoch 1/30, Step: 199/300, total loss:7.2955
Epoch 1/30, Step: 249/300, total loss:3.8893
Epoch 1/30, Step: 299/300, total loss:5.9316
Epoch 1, accuracy: 0.9658, validation loss: 0.0064
Epoch 2/30, Step: 49/300, total loss:4.0534
Epoch 2/30, Step: 99/300, total loss:3.2219
Epoch 2/30, Step: 149/300, total loss:4.1571
Epoch 2/30, Step: 199/300, total loss:3.3835
Epoch 2/30, Step: 249/300, total loss:3.8358
Epoch 2/30, Step: 299/300, total loss:3.5502
Epoch 2, accuracy: 0.9658, validation loss: 0.0058
Epoch 3/30, Step: 49/300, total loss:2.4066
Epoch 3/30, Step: 99/300, total loss:2.4322
Epoch 3/30, Step: 149/300, total loss:3.5365
Epoch 3/30, Step: 199/300, total loss:2.5846
Epoch 3/30, Step: 249/300, total loss:3.2890
Epoch 3/30, Step: 299/300, total loss:2.2471
Epoch 3, accuracy: 0.9658, validation loss: 0.0053
Epoch 4/30, Step: 49/300, total loss:1.609

In [2]:
%pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.5 MB/s eta 0:00:00
